# Sequence to sequence example in Keras (character-level).

Source:
https://keras.io/examples/lstm_seq2seq/

References:
- https://arxiv.org/abs/1409.3215
- https://arxiv.org/abs/1406.1078

This script demonstrates how to implement a basic character-level sequence-to-sequence model. We apply it to translating short English sentences into short Polish sentences, character-by-character. Note that it is fairly unusual to do character-level machine translation, as word-level models are more common in this domain.

### Data download

Lots of neat sentence pairs datasets. http://www.manythings.org/anki/

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np
import re

batch_size = 64  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '../lang_pairs/pol.txt'

Let's read the sentences (input = English, target = Polish),
and extract the character sets for each language.

Target sequences get additional "start sequence" and "end sequence" characters.

In [2]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()
corpus = []
# input_words.add(' ')
# target_words.add(' ')
# input_words.add('\t')
# target_words.add('\t')
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    # line = line.lower()
    input_text, target_text, ignore = line.split('\t') 
    # input_text = re.sub("'", '', input_text)
    # target_text = re.sub("'", '', target_text)
    # input_text = re.sub(",", ' COMMA', input_text)
    # target_text = re.sub(",", ' COMMA', target_text)
    # input_text = re.sub(r'[?|!|.|"|0-9]', r'', input_text)
    # target_text = re.sub(r'[?|!|.|"|0-9]', r'', target_text)

    target_text = 'start_ ' + target_text + ' _end'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for word in input_text.split():
        if word not in input_words:
            input_words.add(word)
    for word in target_text.split():
        if word not in target_words:
            target_words.add(word)
            


In [3]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer1 = CountVectorizer(lowercase=False)
# vectorizer2 = CountVectorizer(lowercase=False)

# X = vectorizer1.fit_transform(input_texts)
# Y = vectorizer2.fit_transform(target_texts)

# input_words = vectorizer1.get_feature_names()
# target_words = vectorizer2.get_feature_names()
# # target_words.append("START_")
# # target_words.append("_END")

# input_words.append("i")
# input_words.append("a")
# input_words.append("COMMA")
# input_words.append(":")
# input_words.append("-")
# input_words.append("b")

# target_words.append("i")
# target_words.append("a")
# target_words.append("w")
# target_words.append("z")
# target_words.append("u")
# target_words.append("o")
# target_words.append("COMMA")
# target_words.append(":")
# target_words.append("-")

In [4]:
# input_words

In [5]:
# target_words

In [6]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt.split()) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 4586
Number of unique output tokens: 8060
Max sequence length for inputs: 6
Max sequence length for outputs: 11


In [7]:
# print(input_words)
# print(target_words)

In [8]:
# print(input_texts)
# print(target_texts)

Make character-to-index dictionaries
and prepare one-hot-encoding of the whole training input data.
Note that the sequences are zero-(post)padded to have all equal lengths.

In [9]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

x_train, y_train = input_texts[:int(len(input_texts)*0.9)], target_texts[:int(len(target_texts)*0.9)]
x_test, y_test = input_texts[int(len(input_texts)*0.9):], target_texts[int(len(target_texts)*0.9):]

encoder_input_data = np.zeros(
    (len(x_train), max_encoder_seq_length),
    dtype='float32')
encoder_test_data = np.zeros(
    (len(x_test), max_encoder_seq_length),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

missed_e = 0
missed_d = 0
for i, (input_text, target_text) in enumerate(zip(x_train, y_train)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]   
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [10]:
for i, (input_text, target_text) in enumerate(zip(x_test, y_test)):
    for t, word in enumerate(input_text.split()):
        encoder_test_data[i, t] = input_token_index[word]

- We start with input sequences from a domain (e.g. English sentences) and corresponding target sequences from another domain (e.g. Polish sentences).
- An encoder LSTM turns input sequences to 2 state vectors (we keep the last LSTM state and discard the outputs).

In [11]:
embedding_size = 50

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size, input_length=max_encoder_seq_length)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

A decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future, a training process called "teacher forcing" in this context. It uses as initial state the state vectors from the encoder. Effectively, the decoder learns to generate `targets[t+1...]` given `targets[...t]`, conditioned on the input sequence.

In [13]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex= dex(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 50)     229300      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     403000      input_2[0][0]                    
______________________________________________________________________________________________

In [16]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05)
# Save model
model.save('s2s-w.h5')

Epoch 1/200
134/134 [==============================] - 25s 165ms/step - loss: 2.1129 - accuracy: 0.0941 - val_loss: 2.6857 - val_accuracy: 0.0915
Epoch 2/200
134/134 [==============================] - 21s 155ms/step - loss: 1.9771 - accuracy: 0.1005 - val_loss: 2.7529 - val_accuracy: 0.0931
Epoch 3/200
134/134 [==============================] - 20s 150ms/step - loss: 1.9026 - accuracy: 0.1054 - val_loss: 2.7599 - val_accuracy: 0.1000
Epoch 4/200
134/134 [==============================] - 19s 144ms/step - loss: 1.8477 - accuracy: 0.1107 - val_loss: 2.7624 - val_accuracy: 0.0964
Epoch 5/200
134/134 [==============================] - 19s 142ms/step - loss: 1.7986 - accuracy: 0.1141 - val_loss: 2.7522 - val_accuracy: 0.0964
Epoch 6/200
134/134 [==============================] - 20s 149ms/step - loss: 1.7650 - accuracy: 0.1153 - val_loss: 2.7471 - val_accuracy: 0.1022
Epoch 7/200
134/134 [==============================] - 20s 148ms/step - loss: 1.7369 - accuracy: 0.1171 - val_loss: 2.7248 -

In inference mode, when we want to decode unknown input sequences, we:
- Encode the input sequence into state vectors and retrieve initial decoder state
- Start with a target sequence of size 1 (just the start-of-sequence character)
- Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character
- Sample the next character using these predictions (we simply use argmax).
- Append the sampled character to the target sequence
- Repeat until we generate the end-of-sequence character or we hit the character limit.

In [17]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          229300    
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 314368    
Total params: 543,668
Trainable params: 543,668
Non-trainable params: 0
_________________________________________________________________


In [18]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
final_dex2= dex(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, word) for word, i in target_token_index.items())

In [19]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['start_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_end' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [20]:
# print(target_token_index)

In [21]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    # print('Input seq:', input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence:  Idź. _end
-
Input sentence: Hi.
Decoded sentence:  Cześć. _end
-
Input sentence: Run!
Decoded sentence:  Uciekaj! _end
-
Input sentence: Run.
Decoded sentence:  Naprawdę? _end
-
Input sentence: Run.
Decoded sentence:  Naprawdę? _end
-
Input sentence: Who?
Decoded sentence:  Kto? _end
-
Input sentence: Wow!
Decoded sentence:  Łał! _end
-
Input sentence: Wow!
Decoded sentence:  Łał! _end
-
Input sentence: Duck!
Decoded sentence:  Unik! _end
-
Input sentence: Fire!
Decoded sentence:  Strzelaj! _end


In [22]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_test_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    # print('Input seq:', input_seq)
    print('Input sentence:', x_test[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: It started to snow.
Decoded sentence:  To wygląda zdrowo.
-
Input sentence: It was a hot night.
Decoded sentence:  To był gorący
-
Input sentence: It was just a joke.
Decoded sentence:  To była tylko
-
Input sentence: It was only a joke.
Decoded sentence:  To była tylko
-
Input sentence: It was quite windy.
Decoded sentence:  To była magia.
-
Input sentence: It was spectacular.
Decoded sentence:  To zależy od
-
Input sentence: It was very simple.
Decoded sentence:  To było bardzo
-
Input sentence: It wasn't my fault.
Decoded sentence:  Nie było mój
-
Input sentence: It wasn't that bad.
Decoded sentence:  Nie było tak
-
Input sentence: It wasn't very fun.
Decoded sentence:  To było moja
